In [30]:
import sys
#sys.path.append('/pl/active/banich/studies/Relevantstudies/abcd/env/lib/python3.7/site-packages')
sys.path.append('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/analysis/')
from functions import *

In [31]:
def get_resids(data, var, formula):
    import statsmodels.formula.api as smf
    model = smf.ols(var + formula, data=data)
    model = model.fit()
    resid = model.resid
    return resid

def add_resid(data, var, formula):
    data[var + "_resid"] = get_resids(data, var, formula)
    return data

full_formula = '~ interview_age + C(sex) + meanmotion'
no_sex_formula = '~ interview_age'

In [32]:
include = pd.read_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/inclusion_ids.csv')

dropped_siblings = pd.read_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/dropped_siblings.csv')

participants_full = (pd.read_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/participants.tsv', sep='\t')
                        .assign(ID = lambda x: x['participant_id'].str.replace('sub-', ''),
                                sex = lambda x: np.where(x['sex'] == 1, 'Male', 'Female'))
                        .drop_duplicates('ID', keep='first')
                    )

#participants_site_scanner_software = participants_full[['ID', 'site', 'scanner_model', 'scanner_software']]

participants = participants_full[['ID', 'matched_group']]
participants_sex = participants_full[['ID', 'sex']]
participants = participants.merge(dropped_siblings, on='ID')
participants = participants.merge(include, on='ID')
participants = participants [['ID', 'matched_group']]

In [33]:
sample1 = participants.query('matched_group == 1')
sample2 = participants.query('matched_group == 2')

males = participants_sex.query('sex == "Male"')
females = participants_sex.query('sex == "Female"')

In [34]:
rpath_subcort = '/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/rest/mrirscor02.txt'


rest_baseline_subcort = readtxt(rpath_subcort)
rest_baseline_subcort['src_subject_id'] = rest_baseline_subcort['src_subject_id'].str.replace('_', '', regex=True)

rest_baseline_subcort_mask = rest_baseline_subcort['eventname'].str.contains('baseline')   
rest_baseline_subcort = rest_baseline_subcort[rest_baseline_subcort_mask]

rest_baseline_subcort = rest_baseline_subcort.drop(['collection_id', 'interview_age', 'mrirscor02_id','dataset_id', 'subjectkey',
                                                    'interview_date', 'eventname','rsfmri_cor_ngd_scs_visitid', 'sex',
                                                    'rsfmri_cor_ngd_scs_tr','rsfmri_cor_ngd_scs_numtrs','rsfmri_cor_ngd_scs_nvols',
                                                    'rsfmri_cor_ngd_scs_stnvols','rsfmri_cor_ngd_scs_stcgnvols','rsfmri_cor_ngd_scs_ntpoints',
                                                    'rsfmri_cor_ngd_scs_meanmn','rsfmri_cor_ngd_scs_maxmn','rsfmri_cor_ngd_scs_meantrans',
                                                    'rsfmri_cor_ngd_scs_maxtrans','rsfmri_cor_ngd_scs_meanrot','rsfmri_cor_ngd_scs_maxrot','collection_title'], axis = 1).rename(columns={"src_subject_id": "ID"})

rest_baseline_subcort.columns = rest_baseline_subcort.columns.str.replace('rsfmri_cor_ngd_', '', regex=True)
rest_baseline_subcort.columns = rest_baseline_subcort.columns.str.replace('_scs_', '_', regex=True) 

rest_baseline_subcort[rest_baseline_subcort.iloc[:, 1:].columns] = rest_baseline_subcort.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

rest_baseline_subcort = pd.merge(rest_baseline_subcort, participants[['ID']], on='ID')

In [35]:
rest_baseline_subcort.shape

(7525, 248)

In [46]:
#rest_creation = '''
rpath = '/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/rest/abcd_betnet02.txt'

rest_baseline = readtxt(rpath)
rest_baseline['src_subject_id'] = rest_baseline['src_subject_id'].str.replace('_', '', regex=True)

rest_1_mask = rest_baseline['rsfmri_c_ngd_visitid'].str.contains('baseline')   
rest_baseline= rest_baseline[rest_1_mask]

rest_motion = rest_baseline[['src_subject_id', 'rsfmri_c_ngd_tr', 'rsfmri_c_ngd_numtrs',
       'rsfmri_c_ngd_nvols', 'rsfmri_c_ngd_stnvols',
       'rsfmri_c_ngd_stcontignvols', 'rsfmri_c_ngd_ntpoints',
       'rsfmri_c_ngd_meanmotion', 'rsfmri_c_ngd_maxmotion',
       'rsfmri_c_ngd_meantrans', 'rsfmri_c_ngd_maxtrans',
       'rsfmri_c_ngd_meanrot', 'rsfmri_c_ngd_maxrot']].rename(columns={"src_subject_id": "ID"})

rest_baseline = rest_baseline.drop(['collection_id', 'abcd_betnet02_id', 'dataset_id', 'subjectkey','interview_date', 'eventname',
       'rsfmri_c_ngd_visitid', 'rsfmri_c_ngd_tr', 'rsfmri_c_ngd_numtrs',
       'rsfmri_c_ngd_nvols', 'rsfmri_c_ngd_stnvols',
       'rsfmri_c_ngd_stcontignvols', 'rsfmri_c_ngd_ntpoints',
       'rsfmri_c_ngd_meanmotion', 'rsfmri_c_ngd_maxmotion',
       'rsfmri_c_ngd_meantrans', 'rsfmri_c_ngd_maxtrans',
       'rsfmri_c_ngd_meanrot', 'rsfmri_c_ngd_maxrot', 'collection_title'], axis = 1).rename(columns={"src_subject_id": "ID"})

rest_baseline.columns = rest_baseline.columns.str.replace('rsfmri_c_ngd_', '', regex=True)
rest_baseline.columns = rest_baseline.columns.str.replace('_ngd_', '_', regex=True) 

rest_cols_subset = ['ID','interview_age', 'sex', 'ad_ad','ad_cgc','ad_ca','ad_dt','ad_dla',
'ad_fo','ad_n','ad_rspltp','ad_smh','ad_smm','ad_sa','ad_vta','ad_vs','cgc_cgc',
'cgc_ca','cgc_dt','cgc_dla','cgc_fo','cgc_n','cgc_rspltp','cgc_smh','cgc_smm','cgc_sa',
'cgc_vta','cgc_vs','ca_ca','ca_dt','ca_dla','ca_fo','ca_n','ca_rspltp','ca_smh','ca_smm',
'ca_sa','ca_vta','ca_vs','dt_dt','dt_dla','dt_fo','dt_n','dt_rspltp','dt_smh','dt_smm','dt_sa','dt_vta',
'dt_vs','dla_dla','dla_fo','dla_n','dla_rspltp','dla_smh','dla_smm','dla_sa','dla_vta',
'dla_vs','fo_fo','fo_n','fo_rspltp','fo_smh','fo_smm','fo_sa','fo_vta','fo_vs','n_n',
'n_rspltp','n_smh','n_smm','n_sa','n_vta','n_vs','rspltp_rspltp','rspltp_smh','rspltp_smm',
'rspltp_sa','rspltp_vta','rspltp_vs','smh_smh','smh_smm','smh_sa','smh_vta','smh_vs',
'smm_smm','smm_sa','smm_vta','smm_vs','sa_sa','sa_vta','sa_vs','vta_vta','vta_vs','vs_vs']

rest_baseline = rest_baseline[rest_cols_subset]

rest_baseline[rest_cols_subset[3:]] = rest_baseline[rest_cols_subset[3:]].apply(pd.to_numeric, errors='coerce')
rest_baseline['interview_age'] = rest_baseline['interview_age'].apply(pd.to_numeric, errors='coerce')
#rest_baseline = pd.merge(rest_baseline, participants_sex, on = 'ID').dropna().drop_duplicates()
rest_baseline = pd.merge(rest_baseline, participants[['ID']], on='ID')

rest_baseline.to_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/rest/rest_baseline_include.csv')
rest_motion.to_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/rest/rest_baseline_motion_include.csv')

In [47]:
rest_motion = pd.read_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/rest/rest_baseline_motion.csv').iloc[:,1:]

rest_motion.columns = rest_motion.columns.str.replace('rsfmri_c_ngd_', '', regex=True)

rest_motion = rest_motion.drop(['tr', 'numtrs', 'nvols', 'stnvols','stcontignvols', 'ntpoints'], axis=1)
#rest_motion = rest_motion.iloc[:, :3]

#sample1_rest_idsub = sample1_rest_idsub.assign(Sample = 'Sample 1')
#sample2_rest_idsub = sample2_rest_idsub.assign(Sample = 'Sample 2')
#sample_rest_idsub = pd.concat([sample1_rest_idsub, sample2_rest_idsub])

#rest_test = pd.merge(sample_rest_idsub, rest_motion, on='ID')
#rest_motion_cols = rest_test.iloc[:, 3:].columns

#rest_melt = rest_test.melt(id_vars=['ID', 'Subtype', 'Sample'])

rest_motion

,ID,meanmotion,maxmotion,meantrans,maxtrans,meanrot,maxrot
0,NDARINVZHZZXCM0,2.747832,32.836193,1.880737,26.050970,0.867095,8.288941
1,NDARINVZJ8TBWKJ,0.270536,7.917863,0.141097,5.528693,0.129439,6.714302
2,NDARINV01D03VR7,0.081854,0.519552,0.051742,0.420261,0.030112,0.156278
3,NDARINV7M5CVFA6,0.090770,2.939569,0.050819,1.196087,0.039950,1.743482
4,NDARINV7XVXWCME,0.313667,9.124249,0.198183,5.995591,0.115484,3.205140
...,...,...,...,...,...,...,...
11309,NDARINVLDK00HZG,0.176993,12.022985,0.112686,4.871183,0.064308,7.151802
11310,NDARINVL5JJEUJJ,0.168726,4.168470,0.117136,3.433667,0.051590,1.031883
11311,NDARINVLVJCXFZ6,0.094247,2.234484,0.061493,1.896588,0.032754,1.170054
11312,NDARINVLNMK5FAN,0.245031,3.331714,0.131590,1.474961,0.113441,1.918795


In [48]:
rest_cols = rest_baseline.iloc[:, 3:].columns.to_list()
rest_baseline = rest_baseline.merge(rest_motion[['ID', 'meanmotion']], on='ID')

In [51]:
rest_baseline = rest_baseline[['ID', 'interview_age', 'sex', 'meanmotion'] + rest_cols]

In [52]:
rest_baseline = pd.merge(rest_baseline, rest_baseline_subcort, on ='ID').dropna()

In [53]:
rest_baseline_resid = rest_baseline.copy()

for i in rest_baseline_resid.drop(['ID', 'sex', 'interview_age', 'meanmotion'], axis=1).columns:
    rest_baseline_resid = add_resid(rest_baseline_resid, i, full_formula)

In [54]:
rest_baseline_resid = pd.concat([rest_baseline_resid[['ID']],rest_baseline_resid.filter(regex='_resid')], axis=1)
rest_baseline_resid.columns = [col.replace('_resid', '') for col in rest_baseline_resid.columns]
#rest_baseline_resid.to_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/rest/rest_baseline_resid_include.csv')

In [55]:
#rest_cols = rest_baseline_resid.iloc[:, 1:].columns.to_list()
#rest_outliers = get_anoms(rest_baseline_resid, rest_cols)
#print(rest_outliers.groupby('outliers_sum').count().iloc[:, :1])
#rest_outliers = rest_outliers.query('outliers_sum != -3').drop('outliers_sum', axis=1)

In [56]:
sample1_rest = pd.merge(rest_baseline_resid, sample1, on='ID').drop('matched_group', axis=1)
sample2_rest = pd.merge(rest_baseline_resid, sample2, on='ID').drop('matched_group', axis=1)
#sample1_rest= pd.merge(rest_outliers, sample1, on='ID').drop('matched_group', axis=1)
#sample2_rest= pd.merge(rest_outliers, sample2, on='ID').drop('matched_group', axis=1)

sample1_rest.to_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/subtype_data/baseline/sample1_rest_baseline_07282023_include_fd.csv')
sample2_rest.to_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/subtype_data/baseline/sample2_rest_baseline_07282023_include_fd.csv')

In [57]:
full_sample = pd.concat([sample1_rest, sample2_rest])

full_sample.to_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/subtype_data/baseline/fullsample_rest_baseline_07282023_include_fd.csv')

In [58]:
full_sample

,ID,ad_ad,ad_cgc,ad_ca,ad_dt,ad_dla,ad_fo,ad_n,ad_rspltp,ad_smh,...,vs_vtdclh,vs_crcxrh,vs_thprh,vs_cderh,vs_ptrh,vs_plrh,vs_hprh,vs_agrh,vs_aarh,vs_vtdcrh
0,NDARINV7XVXWCME,0.000664,-0.045353,-0.033957,0.109161,-0.022622,0.043149,0.042260,0.012099,0.026141,...,0.026304,-0.005916,0.037123,0.014805,-0.135232,-0.010851,-0.059398,-0.083099,-0.014313,-0.093578
1,NDARINV8CNEFJ83,-0.031041,0.035707,-0.059517,-0.080760,0.047413,-0.048485,-0.039752,-0.028513,-0.013173,...,-0.049299,0.044205,-0.015296,-0.044183,-0.045896,0.055047,-0.036063,0.009305,-0.146204,0.031598
2,NDARINV8M2Z9BTA,-0.041531,0.009013,0.022613,-0.010202,0.001500,0.006873,-0.019348,-0.037939,-0.043610,...,-0.006211,0.008741,-0.016541,0.062194,0.013446,-0.040498,0.132414,-0.012785,0.052254,0.003738
3,NDARINVDFY7YYM9,0.024608,-0.004188,-0.040954,0.018301,-0.000179,-0.010191,-0.026724,-0.043921,0.002969,...,-0.017097,-0.010802,-0.009056,0.018407,0.008282,0.099981,0.018136,-0.151919,-0.124374,-0.016441
4,NDARINVE2JXEZ3N,0.080561,0.069851,-0.069913,-0.065404,0.015232,-0.021982,0.017138,-0.060120,-0.092501,...,-0.176250,0.068574,-0.111476,0.058160,0.001537,-0.003959,0.147395,-0.144445,0.006846,0.057103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3618,NDARINVKR1PVY4C,0.030257,-0.122357,0.048002,-0.014735,-0.041927,-0.116081,-0.040435,0.030150,0.118866,...,0.026529,0.055414,0.107007,0.062685,-0.135375,-0.219159,-0.275554,0.268290,0.015941,-0.177579
3619,NDARINVKWG00F0U,-0.006953,0.027083,-0.069949,0.003148,-0.009628,-0.040263,-0.022549,-0.038802,-0.056845,...,0.119579,0.000740,-0.027377,0.059562,0.071696,0.002901,0.023315,0.046316,0.069613,-0.030326
3620,NDARINVKTN3FFWD,0.087472,0.097048,0.024546,0.033229,-0.060987,-0.019811,-0.016693,-0.048407,0.050305,...,0.101397,-0.048040,0.099332,0.091979,0.063202,0.027577,0.027126,0.165338,0.060230,-0.013849
3621,NDARINVL42FE3NR,-0.009332,-0.022523,0.023812,0.062294,0.020957,0.026532,0.033309,0.053590,-0.023728,...,0.119860,-0.055407,0.000951,0.033786,-0.081999,-0.037433,-0.103047,-0.085335,0.007607,-0.032457
